In [1]:
#!pip install -q -U watermark

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import chardet
import sys, os

In [3]:
%reload_ext watermark
%watermark -a "Leonardo da Silva Neves"

Author: Leonardo da Silva Neves



In [4]:
# Carregando DataFrames das edições do censo da educação superior.
censo_23 = 'censup/MICRODADOS_CADASTRO_CURSOS_2023.CSV'
censo_22 = 'censup/MICRODADOS_CADASTRO_CURSOS_2022.CSV'
censo_21 = 'censup/MICRODADOS_CADASTRO_CURSOS_2021.CSV'
censo_20 = 'censup/MICRODADOS_CADASTRO_CURSOS_2020.CSV' 
censo_19 = 'censup/MICRODADOS_CADASTRO_CURSOS_2019.CSV'
censo_18 = 'censup/MICRODADOS_CADASTRO_CURSOS_2018.CSV'
censo_17 = 'censup/MICRODADOS_CADASTRO_CURSOS_2017.CSV'
censo_16 = 'censup/MICRODADOS_CADASTRO_CURSOS_2016.CSV'
censo_15 = 'censup/MICRODADOS_CADASTRO_CURSOS_2015.CSV'
censo_14 = 'censup/MICRODADOS_CADASTRO_CURSOS_2013.CSV'
censo_13 = 'censup/MICRODADOS_CADASTRO_CURSOS_2013.CSV'
censo_12 = 'censup/MICRODADOS_CADASTRO_CURSOS_2012.CSV'
censo_11 = 'censup/MICRODADOS_CADASTRO_CURSOS_2011.CSV'
censo_10 = 'censup/MICRODADOS_CADASTRO_CURSOS_2010.CSV'

In [5]:
# Abrindo o arquivo como tipo binário 'rb' - 'read binary'
with open(censo_20, 'rb') as f1:
    raw_data_23 = f1.read(1000000)

In [6]:
# Descobrindo a codificação do dataset com o método .detect() da biblioteca chardet.
result_23 = chardet.detect(raw_data_23)
cod_23 = result_23['encoding']
conf_23 = result_23['confidence']
print(f'codificação detectada: {cod_23} (confiança: {conf_23:.2f}%)')

codificação detectada: ISO-8859-1 (confiança: 0.73%)


In [7]:
# Carregando o DataFrame em uma variável.
df_23 = pd.read_csv(censo_23, sep=';', encoding=cod_23, low_memory=False, decimal=',')

In [8]:
df_22 = pd.read_csv(censo_22, sep=';', encoding=cod_23, low_memory=False, decimal=',')

In [9]:
df_21 = pd.read_csv(censo_21, sep=';', encoding=cod_23, low_memory=False, decimal=',')

In [10]:
df_20 = pd.read_csv(censo_20, sep=';', encoding=cod_23, low_memory=False, decimal=',')

In [11]:
df_19 = pd.read_csv(censo_19, sep=';', encoding=cod_23, low_memory=False, decimal=',')

In [12]:
df_18 = pd.read_csv(censo_18, sep=';', encoding=cod_23, low_memory=False, decimal=',')

In [13]:
df_17 = pd.read_csv(censo_17, sep=';', encoding=cod_23, low_memory=False, decimal=',')

In [14]:
df_16 = pd.read_csv(censo_16, sep=';', encoding=cod_23, low_memory=False, decimal=',')

In [15]:
df_15 = pd.read_csv(censo_15, sep=';', encoding=cod_23, low_memory=False, decimal=',')

In [16]:
df_14 = pd.read_csv(censo_14, sep=';', encoding=cod_23, low_memory=False, decimal=',')

In [17]:
df_13 = pd.read_csv(censo_13, sep=';', encoding=cod_23, low_memory=False, decimal=',')

In [18]:
df_12 = pd.read_csv(censo_12, sep=';', encoding=cod_23, low_memory=False, decimal=',')

In [19]:
df_11 = pd.read_csv(censo_11, sep=';', encoding=cod_23, low_memory=False, decimal=',')

In [20]:
df_10 = pd.read_csv(censo_10, sep=';', encoding=cod_23, low_memory=False, decimal=',')

### Identificando as colunas que são comuns a todos os DataFrames e, em seguida, manter apenas essas colunas em cada um deles.
Este procedimento garante que, ao concatenar, você não serão mostradas colunas com valores 'NaN'.

In [21]:
# 1.1 Encontrando as colunas comuns
# Carregando os dfs em uma lista
dfs = [df_10, df_11, df_12, df_13, df_14, df_15, df_16, df_17, df_18, df_19, df_20, df_21, df_22, df_23]

In [22]:
# 1.2 Pegando convertendo a lista de colunas de cada df para o tipo set()
colunas_10 = set(dfs[0].columns)
colunas_11 = set(dfs[1].columns)
colunas_12 = set(dfs[2].columns)
colunas_13 = set(dfs[3].columns)
colunas_14 = set(dfs[4].columns)
colunas_15 = set(dfs[5].columns)
colunas_16 = set(dfs[6].columns)
colunas_17 = set(dfs[7].columns)
colunas_18 = set(dfs[8].columns)
colunas_19 = set(dfs[9].columns)
colunas_20 = set(dfs[10].columns)
colunas_21 = set(dfs[11].columns)
colunas_22 = set(dfs[12].columns)
colunas_23 = set(dfs[13].columns)

In [23]:
# Encontra a interseção de todas as colunas
# O resultado será um conjunto de colunas presentes em todos os DataFrames
colunas_comuns = colunas_10.intersection(colunas_11).intersection(colunas_12).intersection(colunas_13).intersection(colunas_14).intersection(colunas_15).intersection(colunas_16).intersection(colunas_17).intersection(colunas_18).intersection(colunas_19).intersection(colunas_20).intersection(colunas_21).intersection(colunas_22).intersection(colunas_23)

In [24]:
# Converte o conjunto de volta para o formato de lista
colunas_comuns_lista = list(colunas_comuns)

In [25]:
print(f"Número total de colunas em 2010: {len(colunas_10)}")
print(f"Número total de colunas em 2011: {len(colunas_11)}")
print(f"Número total de colunas em 2012: {len(colunas_12)}")
print(f"Número total de colunas em 2013: {len(colunas_13)}")
print(f"Número total de colunas em 2014: {len(colunas_14)}")
print(f"Número total de colunas em 2015: {len(colunas_15)}")
print(f"Número total de colunas em 2016: {len(colunas_16)}")
print(f"Número total de colunas em 2017: {len(colunas_17)}")
print(f"Número total de colunas em 2018: {len(colunas_18)}")
print(f"Número total de colunas em 2019: {len(colunas_19)}")
print(f"Número total de colunas em 2020: {len(colunas_20)}")
print(f"Número total de colunas em 2021: {len(colunas_21)}")
print(f"Número total de colunas em 2022: {len(colunas_22)}")
print(f"Número total de colunas em 2023: {len(colunas_23)}")
print(f"Número de colunas comuns {len(colunas_comuns)}")

Número total de colunas em 2010: 200
Número total de colunas em 2011: 200
Número total de colunas em 2012: 200
Número total de colunas em 2013: 200
Número total de colunas em 2014: 200
Número total de colunas em 2015: 200
Número total de colunas em 2016: 200
Número total de colunas em 2017: 200
Número total de colunas em 2018: 200
Número total de colunas em 2019: 200
Número total de colunas em 2020: 200
Número total de colunas em 2021: 200
Número total de colunas em 2022: 200
Número total de colunas em 2023: 202
Número de colunas comuns 199


In [26]:
# Iterando sobre a lista de DataFrames
dfs_padronizados = []
for df_ano in dfs:
    # Selecionar apenas as colunas comuns para cada DataFrame
    df_padronizado = df_ano[colunas_comuns_lista]
    dfs_padronizados.append(df_padronizado)

# Concatenando os DataFrames padronizados em um único DataFrame final
df_completo_padronizado = pd.concat(dfs_padronizados, ignore_index=True)

In [27]:
df_completo_padronizado.shape

(3071995, 199)

In [28]:
# Criando um df apenas com os cursos da modalidade EaD
df_censo_ead = df_completo_padronizado[df_completo_padronizado['TP_MODALIDADE_ENSINO'] == 2].copy()

In [30]:
df_censo_ead.shape

(2606157, 199)

In [32]:
# Substuindo valores da variável 'TP_MODALIDADE_ENSINO'
df_censo_ead['TP_MODALIDADE_ENSINO'] = df_censo_ead['TP_MODALIDADE_ENSINO'].replace(2, 'EaD')

In [33]:
# Alterando o parâmetro máximo de visualizações de colunas
pd.set_option('display.max_columns', 202)

In [34]:
df_censo_ead['NU_ANO_CENSO'] = pd.to_datetime(df_censo_ead['NU_ANO_CENSO'], format='%Y')

In [35]:
# Selecionando apenas colunas com valores numéricos
colunas_quantidade = [col for col in df_censo_ead.columns if col.startswith('QT_')]

In [36]:
# Preechendo os valores nulos (NaN) para evitar erros nas operações matemáricas, usando 0 nas colunas de contagem
df_censo_ead[colunas_quantidade] = df_censo_ead[colunas_quantidade].fillna(0)

In [37]:
df_censo_ead['NO_CINE_AREA_GERAL'].value_counts()

NO_CINE_AREA_GERAL
Negócios, administração e direito                             900467
Educação                                                      654308
Computação e Tecnologias da Informação e Comunicação (TIC)    223147
Saúde e bem-estar                                             206051
Engenharia, produção e construção                             193567
Serviços                                                      167611
Artes e humanidades                                           121959
Ciências sociais, comunicação e informação                     87043
Ciências naturais, matemática e estatística                    28183
Agricultura, silvicultura, pesca e veterinária                 21325
Programas básicos                                               2496
Name: count, dtype: int64

In [38]:
df_censo_ead['NO_CINE_AREA_ESPECIFICA'].value_counts()

NO_CINE_AREA_ESPECIFICA
Negócios e administração                                                                                     866762
Educação                                                                                                     650185
Computação e Tecnologias da Informação e Comunicação (TIC)                                                   220964
Saúde                                                                                                        135731
Engenharia e profissões correlatas                                                                           117670
Artes                                                                                                         72023
Serviços de segurança                                                                                         72015
Bem-estar                                                                                                     68103
Serviços pessoais                               

In [39]:
df_censo_ead['NO_CINE_AREA_DETALHADA'].value_counts()

NO_CINE_AREA_DETALHADA
Gestão e administração                                                     526672
Formação de professores em áreas específicas (exceto Letras)               373903
Formação de professores sem áreas específicas                              131006
Formação de professores de letras                                          127538
Marketing e propaganda                                                     102316
                                                                            ...  
Programas interdisciplinares abrangendo serviços                              382
Mineração e extração                                                          191
Programas abrangendo educação em processo de definição da classificação       136
Medicina                                                                       67
Engenharia e profissões correlatas sem definição precisa                       43
Name: count, Length: 76, dtype: int64

### 1. Métricas de Conclusão e Evasão
Métricas importantes para entender a eficiência e retenção dos cursos.

In [40]:
df_censo_ead['QT_ING'].sum()

18863585.0

In [42]:
df_censo_ead.groupby('NU_ANO_CENSO')['QT_ING'].sum().sort_values(ascending=False)

NU_ANO_CENSO
2023-01-01    3314402.0
2022-01-01    3100556.0
2021-01-01    2477374.0
2020-01-01    2008979.0
2019-01-01    1592184.0
2018-01-01    1373321.0
2017-01-01    1073497.0
2013-01-01    1030820.0
2016-01-01     843181.0
2015-01-01     694559.0
2012-01-01     542696.0
2011-01-01     431631.0
2010-01-01     380385.0
Name: QT_ING, dtype: float64

In [43]:
df_censo_ead.groupby('NU_ANO_CENSO')['QT_MAT'].sum().sort_values(ascending=False)

NU_ANO_CENSO
2023-01-01    4913281.0
2022-01-01    4330934.0
2021-01-01    3716370.0
2020-01-01    3105803.0
2019-01-01    2450265.0
2013-01-01    2307280.0
2018-01-01    2056559.0
2017-01-01    1759250.0
2016-01-01    1494427.0
2015-01-01    1393780.0
2012-01-01    1113958.0
2011-01-01     993062.0
2010-01-01     930920.0
Name: QT_MAT, dtype: float64

In [45]:
df_censo_ead.groupby('NU_ANO_CENSO')['QT_CONC'].sum().sort_values(ascending=False)

NU_ANO_CENSO
2023-01-01    591284.0
2021-01-01    485141.0
2022-01-01    483834.0
2020-01-01    400393.0
2013-01-01    322196.0
2019-01-01    316040.0
2018-01-01    273921.0
2017-01-01    252735.0
2015-01-01    233723.0
2016-01-01    230718.0
2012-01-01    174334.0
2011-01-01    151629.0
2010-01-01    144856.0
Name: QT_CONC, dtype: float64

In [46]:
df_censo_ead.groupby(['NU_ANO_CENSO', 'TP_REDE'])['QT_ING'].sum()

NU_ANO_CENSO  TP_REDE
2010-01-01    1.0          40174.0
              2.0         340211.0
2011-01-01    1.0          34045.0
              2.0         397586.0
2012-01-01    1.0          48527.0
              2.0         494169.0
2013-01-01    1.0              0.0
              2.0              0.0
2015-01-01    1.0          30323.0
              2.0         664236.0
2016-01-01    1.0          24490.0
              2.0         818691.0
2017-01-01    1.0          86965.0
              2.0         986532.0
2018-01-01    1.0          62643.0
              2.0        1310678.0
2019-01-01    1.0          32459.0
              2.0        1559725.0
2020-01-01    1.0          48300.0
              2.0        1960679.0
2021-01-01    1.0          42422.0
              2.0        2434952.0
2022-01-01    1.0          67938.0
              2.0        3032618.0
2023-01-01    1.0          87511.0
              2.0        3226891.0
Name: QT_ING, dtype: float64

In [37]:
df_ead_geral.groupby(['NU_ANO_CENSO', 'NO_REGIAO'])['QT_ING_PRETA'].sum().sort_values(ascending=False)

NU_ANO_CENSO  NO_REGIAO   
2023-01-01    Sudeste         128572
2022-01-01    Sudeste         109540
2021-01-01    Sudeste          77013
2020-01-01    Sudeste          62882
2023-01-01    Nordeste         40171
2022-01-01    Nordeste         37051
2021-01-01    Nordeste         26876
2020-01-01    Nordeste         25095
2023-01-01    Sul              24055
2022-01-01    Sul              23095
2021-01-01    Sul              19575
2023-01-01    Centro-Oeste     18485
2022-01-01    Centro-Oeste     18015
2020-01-01    Sul              15130
2021-01-01    Centro-Oeste     14115
2022-01-01    Norte            11204
2023-01-01    Norte            10853
2020-01-01    Centro-Oeste     10704
2021-01-01    Norte             8612
2020-01-01    Norte             7372
Name: QT_ING_PRETA, dtype: int64

In [41]:
df_ead_geral['TP_GRAU_ACADEMICO'].isna().sum()

1417

In [42]:
df_ead_geral['TP_GRAU_ACADEMICO'].value_counts()

TP_GRAU_ACADEMICO
3.0    919145
1.0    561054
2.0    407716
0.0      1083
Name: count, dtype: int64